In [1]:
import pandas as pd
import geopandas as gpd
from shapely import geometry

In [2]:
reg_path = "../data/boundaries/individual_layers/Regions_iledefrance.geojson"
dept_path = "../data/boundaries/individual_layers/Departements_iledefrance.geojson"
arrs_path = "../data/boundaries/individual_layers/Arrondissements_iledefrance.geojson"
parisarrs_path = "../data/boundaries/individual_layers/Arrondissements_paris.geojson"
comm_path = "../data/boundaries/individual_layers/Communes_iledefrance.geojson"
iris_path = "../data/boundaries/individual_layers/IRIS_iledefrance.geojson"

reg_gdf = gpd.read_file(reg_path).to_crs(epsg=2154)
dept_gdf = gpd.read_file(dept_path).to_crs(epsg=2154)
arrs_gdf = gpd.read_file(arrs_path).to_crs(epsg=2154)
parisarrs_gdf = gpd.read_file(parisarrs_path).to_crs(epsg=2154)
comm_gdf = gpd.read_file(comm_path).to_crs(epsg=2154)
iris_gdf = gpd.read_file(iris_path).to_crs(epsg=2154)

In [3]:
parisarrs_comm_aligned = parisarrs_gdf.copy()

# add any missing columns that exist in comm_gdf but not in parisarrs_gdf
for col in comm_gdf.columns:
    if col not in parisarrs_comm_aligned.columns:
        parisarrs_comm_aligned[col] = pd.NA

# reorder to match comm_gdf exactly
parisarrs_comm_aligned = parisarrs_comm_aligned[comm_gdf.columns]

# Paris department and region codes
parisarrs_comm_aligned["INSEE_DEP"] = "75"
parisarrs_comm_aligned["INSEE_REG"] = "11"

# Arrondissement code comes from INSEE_ARM if available
if "INSEE_ARM" in parisarrs_gdf.columns:
    parisarrs_comm_aligned["INSEE_ARR"] = parisarrs_gdf["INSEE_ARM"]

# For any missing arrondissement code, fill with NA explicitly
parisarrs_comm_aligned["INSEE_ARR"] = parisarrs_comm_aligned["INSEE_ARR"].fillna(pd.NA)

# Optional: set STATUT to "arrondissement"
parisarrs_comm_aligned["STATUT"] = "arrondissement"

# save communes without Paris communes
comm_gdf_no_paris = comm_gdf[comm_gdf["INSEE_DEP"] != "75"].copy()

# save new communes with arrondissements
comm_gdf_merged = gpd.GeoDataFrame(pd.concat([comm_gdf_no_paris, parisarrs_comm_aligned], ignore_index=True), crs=comm_gdf.crs)

# Replace INSEE_COM code with code from arrondissement
comm_gdf_merged.loc[comm_gdf_merged['INSEE_COM'] == "75056", 'INSEE_COM'] = comm_gdf_merged.loc[comm_gdf_merged['INSEE_COM'] == "75056", 'INSEE_ARR']

# set correct crs for future calculations (should already be EPSG:2154 but doublecheck to be sure)
comm_gdf_merged.set_crs(epsg=2154)
# save to new geojson file
comm_gdf_merged.to_file("../data/boundaries/individual_layers/Communes_iledefrance_with_parisarrs.geojson", driver="GeoJSON")

In [4]:
# save all geojsons as single geopackage

# Map variable names to desired layer names
layers = {
    "regions": reg_gdf,
    "departements": dept_gdf,
    "arrondissements": arrs_gdf,
    "communes": comm_gdf_merged,
    "iris": iris_gdf
}

# Loop through and write each as a separate layer into a single geopackage
for i, (layer_name, gdf) in enumerate(layers.items()):
    gdf.to_file("../data/boundaries/admin_polygons_alllayers.gpkg", layer=layer_name, driver="GPKG")


In [5]:
# add ile-de-france region code to every IRIS
iris_with_Reg = iris_gdf.copy()
iris_with_Reg["INSEE_REG"] = "REGION_FXX_0000000000001"

In [6]:
# thankfully commune code is already present in IRIS table, but we can bring over the department code
iris_with_RegDepComm = iris_with_Reg.merge(comm_gdf_merged[["INSEE_COM", "INSEE_DEP"]], on="INSEE_COM", how="left")
# add in the department name
iris_with_RegDepComm = iris_with_RegDepComm.merge(dept_gdf[["INSEE_DEP", "NOM_DEP"]], on="INSEE_DEP", how="left")

# add missing department information for 11 IRISs missing departments
iris_with_RegDepComm.loc[iris_with_RegDepComm["INSEE_COM"] == "93059", ["INSEE_DEP", "NOM_DEP"]] = ["93", "Seine-Saint-Denis"]
iris_with_RegDepComm.loc[iris_with_RegDepComm["INSEE_COM"] == "95282", ["INSEE_DEP", "NOM_DEP"]] = ["95", "Val-d'Oise"]
iris_with_RegDepComm.loc[iris_with_RegDepComm["INSEE_COM"] == "60420", ["INSEE_DEP", "NOM_DEP"]] = ["60", "Oise"]

# rename Commune columns for future joins for higher regional choropleths
iris_with_RegDepComm = iris_with_RegDepComm.rename(columns={"CODE_IRIS": "INSEE_IRIS_ID", "INSEE_COM": "INSEE_COM_ID", "INSEE_DEP": "INSEE_DEP_ID"})

In [7]:
'''
# rename arrondissement ID column to be added to IRIS table
arrs_gdf_forirismerge = arrs_gdf.rename(columns={"ID": "INSEE_ARR_ID", "NOM": "NOM_ARR"})
# # add arrondissement code to each IRIS that are within the arrondissement
iris_with_RegDepCommArr = iris_with_RegDepComm.sjoin(arrs_gdf_forirismerge[["INSEE_ARR_ID", "NOM_ARR", "geometry"]], how="left", predicate="within")
# Keep all original IRIS fields + arrondissement ID (drop index_right column)
iris_with_RegDepCommArr = iris_with_RegDepCommArr.drop(columns=["index_right"])
'''

# rename arrondissement ID column to be added to IRIS table
arrs_gdf_forirismerge = arrs_gdf.rename(columns={"ID": "INSEE_ARR_ID", "NOM": "NOM_ARR"})

# compute intersections between IRIS and arrondissements
iris_arr_intersection = gpd.overlay(iris_with_RegDepComm[["INSEE_IRIS_ID", "geometry"]], arrs_gdf_forirismerge[["INSEE_ARR_ID", "NOM_ARR", "geometry"]], how="intersection")

# measure intersection area and pick the arrondissement covering the largest share of each IRIS
iris_arr_intersection["intersect_area"] = iris_arr_intersection.geometry.area
iris_arr_best = (iris_arr_intersection.sort_values("intersect_area", ascending=False).drop_duplicates(subset="INSEE_IRIS_ID"))

# merge that arrondissement information back into your IRIS dataset
iris_with_RegDepCommArr = iris_with_RegDepComm.merge(iris_arr_best[["INSEE_IRIS_ID", "INSEE_ARR_ID", "NOM_ARR"]], on="INSEE_IRIS_ID", how="left")

C:\Users\theoj\AppData\Roaming\Python\Python313\site-packages\geopandas\tools\overlay.py:358: UserWarning: `keep_geom_type=True` in overlay resulted in 2770 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


In [8]:
# rearrange columns
iris_with_RegDepCommArr =  iris_with_RegDepCommArr[['OBJECTID', 'TYP_IRIS', 'IRIS', 'INSEE_IRIS_ID', 'NOM_IRIS', 'INSEE_COM_ID', 'NOM_COM', 'INSEE_ARR_ID', 'NOM_ARR', 'INSEE_DEP_ID', 'NOM_DEP', 'INSEE_REG', 'geometry']]

In [9]:
# set correct crs for future calculations (should already be EPSG:2154 but doublecheck to be sure)
iris_with_RegDepCommArr.set_crs(epsg=2154)
# compute centroid of each IRIS
iris_centroids = iris_with_RegDepCommArr.copy()
'''
iris_centroids["iris_centroid"] = iris_with_RegDepCommArr.geometry.centroid
iris_centroids = iris_centroids.set_geometry("iris_centroid")
'''
iris_centroids["geometry"] = iris_centroids.geometry.centroid

In [10]:
# output centroids as seperate file
iris_centroids.to_file("../data/boundaries/IRIS_centroids.gpkg", driver="GPKG")

# output final file to use for choropleths
iris_with_RegDepCommArr.to_file("../data/boundaries/IRIS_with_RegDepCommArr.gpkg", driver="GPKG")